## Dependencies

In [3]:
#!pip install tensorflow==2.10 opencv-python matplotlib

In [4]:
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [12]:
# We don't see any GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
assert len(gpus) == 0

## Folder Structures

In [13]:
# Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [14]:
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)